<a href="https://colab.research.google.com/github/teamvictory1/Projects/blob/master/YSO_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Work on the project of customer analytic for YSO challenge


To use this python notebook and kaggle dataset, it is essential to register for Kaggle API. To learn more about Kaggle API, read [here](https://github.com/Kaggle/kaggle-api). This notebook will be regularly updated throughout the duration of the project. In order to run this notebook, click on the Google Colab link.

 For people who are not familiar with Google Colab, please read [here](https://medium.com/deep-learning-turkey/google-colab-free-gpu-tutorial-e113627b9f5d) for more information.

*Install PySpark*

In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://apache.mirror.digitalpacific.com.au/spark/spark-2.3.3/spark-2.3.3-bin-hadoop2.7.tgz
!sudo tar -xf spark-2.3.3-bin-hadoop2.7.tgz
!pip install -q findspark

In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.3.3-bin-hadoop2.7"

In [0]:
import findspark
findspark.init()
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").getOrCreate()

**Testing the data from Kaggle challenge for job recommendation**

*Data Collection*

In [4]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"kienvu145","key":"701362c4f8962b18cb4419a33a068f3d"}'}

In [0]:
!pip install -q kaggle

In [0]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [7]:
! kaggle competitions download -c job-recommendation

 80% 57.0M/71.6M [00:00<00:00, 42.2MB/s]
100% 71.6M/71.6M [00:01<00:00, 69.4MB/s]
  0% 0.00/229k [00:00<?, ?B/s]
100% 229k/229k [00:00<00:00, 72.4MB/s]
 97% 67.0M/69.1M [00:00<00:00, 87.1MB/s]
100% 69.1M/69.1M [00:00<00:00, 126MB/s] 
100% 33.6M/33.6M [00:00<00:00, 67.3MB/s]

 91% 21.0M/23.0M [00:00<00:00, 44.8MB/s]
100% 23.0M/23.0M [00:00<00:00, 76.8MB/s]
  0% 0.00/2.66k [00:00<?, ?B/s]
100% 2.66k/2.66k [00:00<00:00, 2.95MB/s]
100% 17.0M/17.0M [00:00<00:00, 42.9MB/s]
100% 17.0M/17.0M [00:00<00:00, 49.0MB/s]
  0% 0.00/76.2k [00:00<?, ?B/s]
100% 76.2k/76.2k [00:00<00:00, 41.4MB/s]
 30% 5.00M/16.5M [00:00<00:00, 43.9MB/s]
100% 16.5M/16.5M [00:00<00:00, 80.8MB/s]
 62% 5.00M/8.11M [00:00<00:00, 47.5MB/s]
100% 8.11M/8.11M [00:00<00:00, 51.9MB/s]
 99% 661M/671M [00:06<00:00, 171MB/s]
100% 671M/671M [00:06<00:00, 106MB/s]
 99% 664M/671M [00:07<00:00, 91.5MB/s]
100% 671M/671M [00:07<00:00, 88.9MB/s]
  0% 0.00/493 [00:00<?, ?B/s]
100% 493/493 [00:00<00:00, 1.23MB/s]


In [8]:
!ls

apps.tsv	  sample_data			 user_history.tsv
apps.zip	  spark-2.3.3-bin-hadoop2.7	 user_history.zip
jobs.zip	  spark-2.3.3-bin-hadoop2.7.tgz  users.tsv
kaggle.json	  splitjobs.zip			 users.zip
popular_jobs.csv  test_users.tsv		 window_dates.tsv
popular_jobs.py   test_users.zip


**EDA**

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
% matplotlib inline

*App.tsv*

In [0]:
app = pd.read_csv('apps.tsv',delimiter='\t',encoding='utf-8')

In [11]:
app.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1603111 entries, 0 to 1603110
Data columns (total 5 columns):
UserID             1603111 non-null int64
WindowID           1603111 non-null int64
Split              1603111 non-null object
ApplicationDate    1603111 non-null object
JobID              1603111 non-null int64
dtypes: int64(3), object(2)
memory usage: 61.2+ MB


In [12]:
app.describe()

,UserID,WindowID,JobID
count,1.603111e+06,1.603111e+06,1.603111e+06
mean,7.627023e+05,3.643567e+00,5.652615e+05
std,4.225649e+05,2.029082e+00,3.239136e+05
min,7.000000e+00,1.000000e+00,1.000000e+00
25%,3.936690e+05,2.000000e+00,2.805265e+05
50%,7.841790e+05,4.000000e+00,5.678070e+05
75%,1.126113e+06,5.000000e+00,8.498670e+05
max,1.472095e+06,7.000000e+00,1.116313e+06


In [13]:
app.head()

,UserID,WindowID,Split,ApplicationDate,JobID
0,47,1,Train,2012-04-04 15:56:23.537,169528
1,47,1,Train,2012-04-06 01:03:00.003,284009
2,47,1,Train,2012-04-05 02:40:27.753,2121
3,47,1,Train,2012-04-05 02:37:02.673,848187
4,47,1,Train,2012-04-05 22:44:06.653,733748


In [18]:
app[app.isnull().any(axis=1)]

,UserID,WindowID,Split,ApplicationDate,JobID


No missing data detected

In [32]:
app.groupby('UserID')['UserID'].nunique()

UserID
7          1
9          1
13         1
14         1
16         1
18         1
24         1
26         1
44         1
47         1
52         1
56         1
57         1
64         1
67         1
72         1
78         1
79         1
80         1
83         1
101        1
106        1
108        1
112        1
123        1
131        1
132        1
133        1
134        1
138        1
          ..
1471948    1
1471950    1
1471955    1
1471958    1
1471967    1
1471976    1
1471983    1
1471988    1
1471992    1
1471995    1
1471996    1
1471997    1
1471998    1
1472005    1
1472008    1
1472019    1
1472022    1
1472035    1
1472036    1
1472042    1
1472058    1
1472060    1
1472066    1
1472069    1
1472075    1
1472079    1
1472085    1
1472089    1
1472090    1
1472095    1
Name: UserID, Length: 321235, dtype: int64

*Popular_job.csv*

In [0]:
popular_job = pd.read_csv('popular_jobs.csv')

In [34]:
popular_job.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22838 entries, 0 to 22837
Data columns (total 2 columns):
UserId     22838 non-null int64
 JobIds    22762 non-null object
dtypes: int64(1), object(1)
memory usage: 356.9+ KB


In [35]:
popular_job.head()

,UserId,JobIds
0,767,299322 602736 1077691 625281 898514 1035141 65...
1,769,340536 28811 305416 970973 765210 114951 90808...
2,861,384120 1096879 23663 1003185 787775 47052 9343...
3,1006,219766 379509 840434 691160 813087 751038 2980...
4,1192,208362 919831 508443 630792 1075583 797189 327...
